# Kaggle API Processing to download dataset

In [1]:
!pip install kaggle
# Create .kaggle folder in ../root directory
!mkdir ~/.kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Upload kaggle.json => Personal API Token downloaded from kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"akritighosh","key":"6338b709da906e0e129e6ff19fe17260"}'}

In [3]:
# Copy kaggle.json file to created .kaggle folder
!cp kaggle.json ~/.kaggle/
# Change permissions for the file
!chmod 600 ~/.kaggle/kaggle.json

## Download dataset zip 

In [4]:
!kaggle datasets download asdasdasasdas/garbage-classification

 70% 57.0M/82.0M [00:00<00:00, 101MB/s] 
100% 82.0M/82.0M [00:00<00:00, 117MB/s]


## Unzip the compressed dataset

In [ ]:
!unzip garbage-classification.zip -d data

## Delete duplicate image folder

In [7]:
!rm -rf '/content/data/Garbage classification'

# Import necessary libraries

In [8]:
# Interacting with the operating system and perform file management tasks
import shutil
# To create, edit and process dataset from csv files
import pandas as pd
# For mathematical operations and multidimensional arrays
import numpy as np
import math
# For image augmentation
from tensorflow.keras.preprocessing.image import  load_img, img_to_array, ImageDataGenerator
# For creating model
import tensorflow as tf

# Import labelled data for train, validation and test dataset

In [9]:
classes = {1:'glass', 2:'paper',3:'cardboard',4:'plastic',5:'metal',6:'trash'}

In [10]:
df = pd.read_csv('/content/data/one-indexed-files.txt',   header=None, names=['File','Label'], sep=' ')
df_train = pd.read_csv('/content/data/one-indexed-files-notrash_train.txt',   header=None, names=['File','Label'], sep=' ')
df_val = pd.read_csv('/content/data/one-indexed-files-notrash_val.txt',   header=None, names=['File','Label'], sep=' ')
df_test = pd.read_csv('/content/data/one-indexed-files-notrash_test.txt',   header=None, names=['File','Label'], sep=' ')

## Train dataset

In [11]:
# Dataframe info
print(df_train.info())
# Get shape
print('No of rows', df_train.shape[0])
df_train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1768 entries, 0 to 1767
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   File    1768 non-null   object
 1   Label   1768 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 27.8+ KB
None
No of rows 1768


,File,Label
0,cardboard202.jpg,3
1,paper472.jpg,2
2,paper522.jpg,2
3,glass189.jpg,1
4,glass325.jpg,1
...,...,...
1763,cardboard6.jpg,3
1764,glass283.jpg,1
1765,metal335.jpg,5
1766,plastic133.jpg,4


## Validation dataset

In [12]:
# Dataframe info
print(df_val.info())
# Get shape
print('No of rows', df_val.shape[0])
df_val

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   File    328 non-null    object
 1   Label   328 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.2+ KB
None
No of rows 328


,File,Label
0,cardboard114.jpg,3
1,plastic204.jpg,4
2,glass123.jpg,1
3,glass152.jpg,1
4,glass398.jpg,1
...,...,...
323,metal58.jpg,5
324,cardboard149.jpg,3
325,plastic5.jpg,4
326,glass23.jpg,1


## Test dataset

In [13]:
# Dataframe info
print(df_test.info())
# Get shape
print('No of rows', df_test.shape[0])
df_test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   File    431 non-null    object
 1   Label   431 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.9+ KB
None
No of rows 431


,File,Label
0,paper70.jpg,2
1,paper380.jpg,2
2,cardboard31.jpg,3
3,glass12.jpg,1
4,paper169.jpg,2
...,...,...
426,metal389.jpg,5
427,paper303.jpg,2
428,paper405.jpg,2
429,paper465.jpg,2


# Create the dataset as per requirement


### Current format of image dataset : -
data / <br/>
. class 1/ <br/>
.... img1 <br/>
.... img2 <br/>
. class 2/ <br/>
. class 3/ <br/>
. class 4/ <br/>


### Format required - 
data / <br/>
.. train / <br/>
.... class 1/ <br/>
....... img1 <br/>
....... img2 <br/>
.... class 2/ <br/>
.... class 3/ <br/>
.... class 4/ <br/>
.. test / <br/>
.... class 1/ <br/>
....... img1 <br/>
....... img2 <br/>
.... class 2/ <br/>
.... class 3/ <br/>
.... class 4/ <br/>


## Train directory

In [14]:
!mkdir '/content/data/train'
!mkdir '/content/data/train/glass'
!mkdir '/content/data/train/paper'
!mkdir '/content/data/train/cardboard'
!mkdir '/content/data/train/plastic'
!mkdir '/content/data/train/metal'
!mkdir '/content/data/train/trash'

In [15]:
for i in range(df_train.shape[0]):
  try:
    folder = classes[int(df_train.iloc[i,1])]
    new_path = '/content/data/train/' + folder
    # old_path = '/content/data/garbage classification/Garbage classification/', folder , str(df_train.iloc[i,0]))
    old_path = '/content/data/garbage classification/Garbage classification/' + folder + '/' + str(df_train.iloc[i,0])
    shutil.move(old_path, new_path)
  except:
    print(i, str(df_train.iloc[i,0]))

## Validation directory

In [16]:
!mkdir '/content/data/val'
!mkdir '/content/data/val/glass'
!mkdir '/content/data/val/paper'
!mkdir '/content/data/val/cardboard'
!mkdir '/content/data/val/plastic'
!mkdir '/content/data/val/metal'
!mkdir '/content/data/val/trash'

In [17]:
for i in range(df_val.shape[0]):
  try:
    folder = classes[int(df_val.iloc[i,1])]
    new_path = '/content/data/val/' + folder
    # old_path = '/content/data/garbage classification/Garbage classification/', folder , str(df_val.iloc[i,0]))
    old_path = '/content/data/garbage classification/Garbage classification/' + folder + '/' + str(df_val.iloc[i,0])
    shutil.move(old_path, new_path)
  except:
    print(i, str(df_val.iloc[i,0]))

## Test directory

In [18]:
!mkdir '/content/data/test'
!mkdir '/content/data/test/glass'
!mkdir '/content/data/test/paper'
!mkdir '/content/data/test/cardboard'
!mkdir '/content/data/test/plastic'
!mkdir '/content/data/test/metal'
!mkdir '/content/data/test/trash'

In [19]:
for i in range(df_test.shape[0]):
  try:
    folder = classes[int(df_test.iloc[i,1])]
    new_path = '/content/data/test/' + folder
    # old_path = '/content/data/garbage classification/Garbage classification/', folder , str(df_test.iloc[i,0]))
    old_path = '/content/data/garbage classification/Garbage classification/' + folder + '/' + str(df_test.iloc[i,0])
    shutil.move(old_path, new_path)
  except:
    print(i, str(df_test.iloc[i,0]))

## Delete old directory

In [20]:
!rm -rf '/content/data/garbage classification'

# Data Augmentation

In [21]:
data_gen = ImageDataGenerator(
        rotation_range=45,
        # width_shift_range=0.2,
        # height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        # brightness_range=[0.75,1.25],
        rescale=1./255,
        # zoom_range=0.25,
        # fill_mode='nearest'
        )

test_gen = ImageDataGenerator(rescale=1./255)

In [22]:
train_data_gen = data_gen.flow_from_directory(
                                        batch_size=128,                           # number of images in each batch
                                        directory='/content/data/train',          # the source of your images
                                        shuffle=True,                             # the images will be shuffle
                                        target_size=(128, 128),                   # size (height, width) of your images
                                        class_mode='categorical'                  # categorical because the dataset contains more than 2 labels
                                        )
val_data_gen = data_gen.flow_from_directory(
                                        batch_size=128,                           # number of images in each batch
                                        directory='/content/data/val',            # the source of your images
                                        shuffle=True,                             # the images will be shuffle
                                        target_size=(128, 128),                   # size (height, width) of your images
                                        class_mode='categorical'                  # categorical because the dataset contains more than 2 labels
                                        )
val_test_gen = test_gen.flow_from_directory(
                                        batch_size=128,                           # number of images in each batch
                                        directory='/content/data/val',            # the source of your images
                                        shuffle=True,                             # the images will be shuffle
                                        target_size=(128, 128),                   # size (height, width) of your images
                                        class_mode='categorical'                  # categorical because the dataset contains more than 2 labels
                                        )
test_data_gen = test_gen.flow_from_directory(
                                        batch_size=128,                           # number of images in each batch
                                        directory='/content/data/test',           # the source of your images
                                        shuffle=True,                             # the images will be shuffle
                                        target_size=(128, 128),                   # size (height, width) of your images
                                        class_mode='categorical'                  # categorical because the dataset contains more than 2 labels
                                        )

Found 1768 images belonging to 6 classes.
Found 328 images belonging to 6 classes.
Found 328 images belonging to 6 classes.
Found 431 images belonging to 6 classes.


# Model Training

* Validation - val_data_gen
* 1 conv layer - 16 with batch normalization and dropout 
* 2 32 dense layers with batch normalization

In [39]:
m_best = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape =(128,128,3)),
                                    tf.keras.layers.BatchNormalization(),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Dropout(0.5),

                                    tf.keras.layers.Flatten(),
                                    # tf.keras.layers.Dense(64, activation='relu'),
                                    tf.keras.layers.Dense(1024, activation='relu'),
                                    tf.keras.layers.Dense(1024, activation='relu'),
                                    tf.keras.layers.Dense(6, activation='softmax')
])


In [40]:
# decrt = 0.1/120
# m_best.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate = 0.1, decay = decrt), metrics=['accuracy'])
m_best.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate = 0.00001), metrics=['accuracy'])
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, restore_best_weights=True)

m_best.fit(
  train_data_gen,
  batch_size = 32, 
  callbacks=[es], 
  validation_data=val_test_gen, 
  epochs=120
)

Epoch 1/120
14/14 [==============================] - 12s 840ms/step - loss: 1.7302 - accuracy: 0.3326 - val_loss: 1.7023 - val_accuracy: 0.2652
Epoch 2/120
14/14 [==============================] - 11s 781ms/step - loss: 1.4969 - accuracy: 0.4406 - val_loss: 1.6917 - val_accuracy: 0.2652
Epoch 3/120
14/14 [==============================] - 11s 791ms/step - loss: 1.3830 - accuracy: 0.4655 - val_loss: 1.6907 - val_accuracy: 0.2744
Epoch 4/120
14/14 [==============================] - 11s 795ms/step - loss: 1.3378 - accuracy: 0.4774 - val_loss: 1.6915 - val_accuracy: 0.2835
Epoch 5/120
14/14 [==============================] - 12s 836ms/step - loss: 1.2952 - accuracy: 0.5164 - val_loss: 1.6722 - val_accuracy: 0.2896
Epoch 6/120
14/14 [==============================] - 11s 785ms/step - loss: 1.2538 - accuracy: 0.5204 - val_loss: 1.6685 - val_accuracy: 0.2744
Epoch 7/120
14/14 [==============================] - 11s 783ms/step - loss: 1.2363 - accuracy: 0.5345 - val_loss: 1.6628 - val_accuracy:

In [41]:
m_best.fit(
  train_data_gen,
  batch_size = 160,
  callbacks=[es], 
  validation_data=val_test_gen, 
  epochs=120
)

Epoch 1/120
14/14 [==============================] - 12s 827ms/step - loss: 0.7155 - accuracy: 0.7472 - val_loss: 0.9873 - val_accuracy: 0.6555
Epoch 2/120
14/14 [==============================] - 11s 796ms/step - loss: 0.7153 - accuracy: 0.7432 - val_loss: 1.0005 - val_accuracy: 0.6677
Epoch 3/120
14/14 [==============================] - 11s 768ms/step - loss: 0.6994 - accuracy: 0.7562 - val_loss: 0.9883 - val_accuracy: 0.6555
Epoch 4/120
14/14 [==============================] - 11s 772ms/step - loss: 0.7022 - accuracy: 0.7489 - val_loss: 1.0130 - val_accuracy: 0.6585
Epoch 5/120
14/14 [==============================] - 11s 795ms/step - loss: 0.6985 - accuracy: 0.7517 - val_loss: 0.9713 - val_accuracy: 0.6646
Epoch 6/120
14/14 [==============================] - 11s 813ms/step - loss: 0.6770 - accuracy: 0.7545 - val_loss: 0.9974 - val_accuracy: 0.6585
Epoch 7/120
14/14 [==============================] - 11s 777ms/step - loss: 0.7036 - accuracy: 0.7483 - val_loss: 1.0053 - val_accuracy:

In [42]:
m_best.fit(
  train_data_gen,
  batch_size = 64,
  callbacks=[es], 
  validation_data=val_test_gen, 
  epochs=120
)

Epoch 1/120
14/14 [==============================] - 12s 855ms/step - loss: 0.6625 - accuracy: 0.7511 - val_loss: 1.0028 - val_accuracy: 0.6860
Epoch 2/120
14/14 [==============================] - 11s 795ms/step - loss: 0.6330 - accuracy: 0.7760 - val_loss: 1.0050 - val_accuracy: 0.6738
Epoch 3/120
14/14 [==============================] - 11s 808ms/step - loss: 0.6460 - accuracy: 0.7664 - val_loss: 0.9889 - val_accuracy: 0.6707
Epoch 4/120
14/14 [==============================] - 11s 788ms/step - loss: 0.6542 - accuracy: 0.7607 - val_loss: 0.9778 - val_accuracy: 0.6646
Epoch 5/120
14/14 [==============================] - 11s 787ms/step - loss: 0.6442 - accuracy: 0.7568 - val_loss: 0.9540 - val_accuracy: 0.6829
Epoch 6/120
14/14 [==============================] - 11s 781ms/step - loss: 0.6336 - accuracy: 0.7562 - val_loss: 0.9671 - val_accuracy: 0.6768
Epoch 7/120
14/14 [==============================] - 11s 796ms/step - loss: 0.6309 - accuracy: 0.7760 - val_loss: 0.9648 - val_accuracy:

In [27]:
m_best.save('/content/78val74train.h5')
files.download('/content/78val74train.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:
from keras.applications.vgg16 import VGG16
from keras.models import Model

In [37]:
conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=(128,128,3))
for layer in conv_base.layers:
  layer.trainable = False
top_model = conv_base.output
top_model = tf.keras.layers.Flatten(name="flatten")(top_model)
top_model = tf.keras.layers.Dense(4096, activation='relu')(top_model)
top_model = tf.keras.layers.Dense(1072, activation='relu')(top_model)
top_model = tf.keras.layers.Dropout(0.2)(top_model)
output_layer = tf.keras.layers.Dense(6, activation='softmax')(top_model)

# Group the convolutional base and new fully-connected layers into a Model object.
model = Model(inputs=conv_base.input, outputs=output_layer)

# Compiles the model for training.
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
vgg_history = model.fit(train_data_gen, batch_size = 64, callbacks=[es], validation_data=val_test_gen, epochs=120)

Epoch 1/120
14/14 [==============================] - 22s 1s/step - loss: 5.2879 - accuracy: 0.2924 - val_loss: 1.5025 - val_accuracy: 0.4451
Epoch 2/120
14/14 [==============================] - 12s 874ms/step - loss: 1.3120 - accuracy: 0.4734 - val_loss: 1.1173 - val_accuracy: 0.5366
Epoch 3/120
14/14 [==============================] - 13s 903ms/step - loss: 1.0561 - accuracy: 0.5786 - val_loss: 0.9448 - val_accuracy: 0.6311
Epoch 4/120
14/14 [==============================] - 12s 870ms/step - loss: 0.9207 - accuracy: 0.6578 - val_loss: 0.8461 - val_accuracy: 0.6799
Epoch 5/120
14/14 [==============================] - 12s 869ms/step - loss: 0.8277 - accuracy: 0.6906 - val_loss: 0.8551 - val_accuracy: 0.6860
Epoch 6/120
14/14 [==============================] - 12s 864ms/step - loss: 0.7377 - accuracy: 0.7172 - val_loss: 0.7269 - val_accuracy: 0.7195
Epoch 7/120
14/14 [==============================] - 13s 916ms/step - loss: 0.6936 - accuracy: 0.7443 - val_loss: 0.7166 - val_accuracy: 0.

In [38]:
model.save('/content/model.h5')
files.download('/content/model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>